# Communication schemes

In [1]:
import os, sys
import numpy as np

# If this is not done before the cunqa module imports, it will fail to find them
sys.path.append(os.getenv("HOME"))

from cunqa import get_QPUs, qraise, gather, qdrop
from cunqa.qpu import run
from cunqa.circuit import CunqaCircuit

## No-communication scheme

In [2]:
n_family = qraise(2, "00:10:00", simulator = "Aer",  co_located = True)
n_qpus = get_QPUs(co_located = True, family = n_family)

qc = CunqaCircuit(2)

theta1 = np.pi / 4
theta2 = np.pi / 3
theta3 = 2 * np.pi / 3

qc.ry(theta1, 0)
qc.x(0)
qc.cry(theta2, 0, 1)
qc.x(0)
qc.measure_all()

n_qjobs = []
for qpu in n_qpus: 
    n_qjobs.append(run(qc, qpu, shots = 1000))

print("Waiting for the results...")
results = gather(n_qjobs)
qdrop(n_family)

Requested QPUs with command:
	qraise -n 2 -t 00:10:00 --simulator=Aer --co-located
QPUs ready to work ✅
Waiting for the results...


RuntimeError: Error during simulation, please check availability of QPUs, run arguments syntax and circuit syntax: Invalid Qobj experiment: not enough memory slots.

## Classical-communication scheme

In [ ]:
c_family = qraise(2, "00:10:00", simulator = "Aer",  co_located = True, classical_comm=True)
c_qpus = get_QPUs(co_located = True, family = c_family)

In [ ]:
########## First circuit ############
cc_1 = CunqaCircuit(10, 2, id="First")
cc_1.h(0)
cc_1.measure_and_send(qubit = 0, target_circuit = "Second")
cc_1.measure(0,0)
cc_1.measure(1,1)

########## Second circuit ###########
cc_2 = CunqaCircuit(2, 2, id="Second")
cc_2.remote_c_if("x", qubits = 0, param=None, control_circuit = "First")
cc_2.measure(0,0)
cc_2.measure(1,1)

In [ ]:
circs = [cc_1, cc_2]
c_qjobs = run(circs, c_qpus, shots=1000) 
result_list = gather(c_qjobs)
for result in result_list:
    print(result)
qdrop(c_family)

## Quantum-communication scheme

In [ ]:
q_family = qraise(2, "00:10:00", simulator = "Aer",  co_located = True, quantum_comm=True)
q_qpus = get_QPUs(co_located = True, family = c_family)

In [ ]:
circuit1 = CunqaCircuit(2, id = "circuit1") # adding ancilla
circuit1.h(0)
circuit1.cx(0,1)
circuit1.qsend(1, "circuit2")# this qubit that is sent is reset
circuit1.measure_all()

circuit2 = CunqaCircuit(1, id = "circuit2")
circuit2.qrecv(0, "circuit1")
circuit2.measure_all()

In [ ]:
qjobs = run([circuit1, circuit2], q_qpus, shots = 1000)
results = gather(qjobs)
for q in results:
    print("Result: ", q.counts)
    print()

qdrop(q_family)